# Create data for Zpl models

## Basic data

In [1]:
from api_fpl.requests import api

In [2]:
fpl = api()

In [19]:
fpl._players[0]

{'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'code': 80201,
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -5,
 'cost_change_start_fall': 5,
 'dreamteam_count': 1,
 'element_type': 1,
 'ep_next': '1.0',
 'ep_this': '0.5',
 'event_points': 0,
 'first_name': 'Bernd',
 'form': '0.0',
 'id': 1,
 'in_dreamteam': False,
 'news': '',
 'news_added': '2022-02-11T08:00:15.144286Z',
 'now_cost': 45,
 'photo': '80201.jpg',
 'points_per_game': '2.5',
 'second_name': 'Leno',
 'selected_by_percent': '0.8',
 'special': False,
 'squad_number': None,
 'status': 'a',
 'team': 1,
 'team_code': 3,
 'total_points': 10,
 'transfers_in': 80974,
 'transfers_in_event': 0,
 'transfers_out': 205566,
 'transfers_out_event': 0,
 'value_form': '0.0',
 'value_season': '2.2',
 'web_name': 'Leno',
 'minutes': 360,
 'goals_scored': 0,
 'assists': 0,
 'clean_sheets': 1,
 'goals_conceded': 9,
 'own_goals': 0,
 'penalties_saved': 0,
 'penalties_missed': 0,
 'yel

--------------------------

In [21]:
teams_file = open("models/data/teams.dat", "w")
teams = {}
for team in fpl._teams:
    teams[team["id"]] = team["name"]
    teams_file.write(f'{team["name"]}\n')
    
teams_file.close()

In [29]:
import unicodedata
import re

def limpiar_texto(input):
    input = unicodedata.normalize('NFD', input)
    input = input.encode('ascii', 'ignore')
    input = input.decode("utf-8")
    input = re.sub('[^\w\s]', '', input).strip()
    return str(input)

In [31]:
scores_file = open("models/data/players_season_predicted_scores.dat", "w")
player_team = open("models/data/players_teams.dat", "w")
costs_file = open("models/data/players_costs.dat", "w")

for player in fpl._players:
    player_name = limpiar_texto(f"{player['first_name'].replace(' ', '_')}_{player['second_name'].replace(' ', '_')}")
    
    team = teams[player["team"]]
    info = f"{player_name}\t{team}"
    player_team.write(f"{info}\n")
    costs_file.write(f"{player_name}\t{player['now_cost']/10}\n")
    
    player_history = fpl._request("player", player["id"])["history"]
    points = []
    for gw in player_history:
        points.append(str(gw["total_points"]))    
    if len(points) < 38:
        missing = 38 - len(points)
        points = ["0"] * missing + points
    scores_file.write(f"{' '.join(points)}\n")
    
    
scores_file.close()
player_team.close()
costs_file.close()
print("Termine!")

Termine!


In [23]:
players = {
    "name" : 
}
for player in fpl._players:

{'fixtures': [],
 'history': [{'element': 1,
   'fixture': 1,
   'opponent_team': 3,
   'total_points': 1,
   'was_home': False,
   'kickoff_time': '2021-08-13T19:00:00Z',
   'team_h_score': 2,
   'team_a_score': 0,
   'round': 1,
   'minutes': 90,
   'goals_scored': 0,
   'assists': 0,
   'clean_sheets': 0,
   'goals_conceded': 2,
   'own_goals': 0,
   'penalties_saved': 0,
   'penalties_missed': 0,
   'yellow_cards': 0,
   'red_cards': 0,
   'saves': 1,
   'bonus': 0,
   'bps': 11,
   'influence': '11.8',
   'creativity': '0.0',
   'threat': '0.0',
   'ict_index': '1.2',
   'value': 50,
   'transfers_balance': 0,
   'selected': 153122,
   'transfers_in': 0,
   'transfers_out': 0},
  {'element': 1,
   'fixture': 18,
   'opponent_team': 6,
   'total_points': 2,
   'was_home': True,
   'kickoff_time': '2021-08-22T15:30:00Z',
   'team_h_score': 0,
   'team_a_score': 2,
   'round': 2,
   'minutes': 90,
   'goals_scored': 0,
   'assists': 0,
   'clean_sheets': 0,
   'goals_conceded': 2,
  

In [3]:
opt = optimizer(fpl)

## Descriptive gameweek model

In [3]:
result = fpl._request("base")

In [5]:
import unicodedata
import re

def limpiar_texto(input):
    input = unicodedata.normalize('NFD', input)
    input = input.encode('ascii', 'ignore')
    input = input.decode("utf-8")
    input = re.sub('[^\w\s]', '', input).strip()
    return str(input)

costs_file = open("models/data/players_costs.dat", "w")
scores_file = open("models/data/players_predicted_scores.dat", "w")
goalkeepers = open("models/data/goalkeepers.dat", "w")
defenders = open("models/data/defenders.dat", "w")
midfielders = open("models/data/midfielders.dat", "w")
forwards = open("models/data/forwards.dat", "w")

teams_files = {}
for team in result["teams"]:
    teams_files[team["code"]] = open(f"models/data/{team['name']}_players.dat", "w")
    
exceptions = 0
for player in result["elements"]:
    player_name = limpiar_texto(f"{player['first_name'].replace(' ', '_')}_{player['second_name'].replace(' ', '_')}")
    try:
        teams_files[player["team_code"]].write(f"{player_name}\n")
        
        costs_file.write(f"{player_name}\t{player['now_cost']/10}\n")
        scores_file.write(f"{player_name}\t{player['event_points']}\n")
        
        if player["element_type"] == 1:
            goalkeepers.write(f"{player_name}\n")
        elif player["element_type"] == 2:
            defenders.write(f"{player_name}\n")
        elif player["element_type"] == 3:
            midfielders.write(f"{player_name}\n")
        elif player["element_type"] == 4:
            forwards.write(f"{player_name}\n")
    except:
        exceptions += 1

costs_file.close()
scores_file.close()
goalkeepers.close()
defenders.close()
midfielders.close()
forwards.close()

for team in result["teams"]:
    teams_files[team["code"]].close()
        
print(f"Total exceptions: {exceptions}")

Total exceptions: 0


## Descriptive season static

In [6]:
fixtures = fpl._request("fixtures")
gameweeks = fpl._request("base")["events"]
players = fpl._request("base")["elements"]

In [7]:
players[0]

{'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'code': 80201,
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -5,
 'cost_change_start_fall': 5,
 'dreamteam_count': 1,
 'element_type': 1,
 'ep_next': '1.0',
 'ep_this': '1.0',
 'event_points': 0,
 'first_name': 'Bernd',
 'form': '0.0',
 'id': 1,
 'in_dreamteam': False,
 'news': '',
 'news_added': '2022-02-11T08:00:15.144286Z',
 'now_cost': 45,
 'photo': '80201.jpg',
 'points_per_game': '2.5',
 'second_name': 'Leno',
 'selected_by_percent': '0.8',
 'special': False,
 'squad_number': None,
 'status': 'a',
 'team': 1,
 'team_code': 3,
 'total_points': 10,
 'transfers_in': 80974,
 'transfers_in_event': 0,
 'transfers_out': 205566,
 'transfers_out_event': 0,
 'value_form': '0.0',
 'value_season': '2.2',
 'web_name': 'Leno',
 'minutes': 360,
 'goals_scored': 0,
 'assists': 0,
 'clean_sheets': 1,
 'goals_conceded': 9,
 'own_goals': 0,
 'penalties_saved': 0,
 'penalties_missed': 0,
 'yel

In [5]:
result = fpl._request("player", 77)
print(result.keys())

dict_keys(['fixtures', 'history', 'history_past'])


In [6]:
result["history"]

[{'element': 77,
  'fixture': 1,
  'opponent_team': 1,
  'total_points': 8,
  'was_home': True,
  'kickoff_time': '2021-08-13T19:00:00Z',
  'team_h_score': 2,
  'team_a_score': 0,
  'round': 1,
  'minutes': 90,
  'goals_scored': 1,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 0,
  'bonus': 0,
  'bps': 25,
  'influence': '38.4',
  'creativity': '1.8',
  'threat': '40.0',
  'ict_index': '8.0',
  'value': 50,
  'transfers_balance': 0,
  'selected': 13090,
  'transfers_in': 0,
  'transfers_out': 0},
 {'element': 77,
  'fixture': 13,
  'opponent_team': 7,
  'total_points': 2,
  'was_home': False,
  'kickoff_time': '2021-08-21T14:00:00Z',
  'team_h_score': 0,
  'team_a_score': 0,
  'round': 2,
  'minutes': 90,
  'goals_scored': 0,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yel

In [10]:
result["elements"][83]

{'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'code': 128295,
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': 0,
 'cost_change_start_fall': 0,
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '0.3',
 'ep_this': '1.3',
 'event_points': 1,
 'first_name': 'Christian',
 'form': '1.8',
 'id': 77,
 'in_dreamteam': False,
 'news': '',
 'news_added': '2022-03-06T00:00:12.205684Z',
 'now_cost': 50,
 'photo': '128295.jpg',
 'points_per_game': '2.5',
 'second_name': 'Nørgaard',
 'selected_by_percent': '0.6',
 'special': False,
 'squad_number': None,
 'status': 'a',
 'team': 3,
 'team_code': 94,
 'total_points': 73,
 'transfers_in': 195662,
 'transfers_in_event': 86,
 'transfers_out': 198133,
 'transfers_out_event': 297,
 'value_form': '0.4',
 'value_season': '14.6',
 'web_name': 'Nørgaard',
 'minutes': 2534,
 'goals_scored': 2,
 'assists': 2,
 'clean_sheets': 6,
 'goals_conceded': 45,
 'own_goals': 0,
 'penalties_saved': 0,
 'penal

In [7]:
base = fpl._request("base")
base["events"][0]

{'id': 1,
 'name': 'Gameweek 1',
 'deadline_time': '2021-08-13T17:30:00Z',
 'average_entry_score': 69,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 5059647,
 'deadline_time_epoch': 1628875800,
 'deadline_time_game_offset': 0,
 'highest_score': 150,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'cup_leagues_created': False,
 'h2h_ko_matches_created': False,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 145658},
  {'chip_name': '3xc', 'num_played': 225749}],
 'most_selected': 275,
 'most_transferred_in': 1,
 'top_element': 277,
 'top_element_info': {'id': 277, 'points': 20},
 'transfers_made': 0,
 'most_captained': 233,
 'most_vice_captained': 277}

In [8]:
fixtures = fpl._request("fixtures")
fixtures[0]

{'code': 2210271,
 'event': 1,
 'finished': True,
 'finished_provisional': True,
 'id': 1,
 'kickoff_time': '2021-08-13T19:00:00Z',
 'minutes': 90,
 'provisional_start_time': False,
 'started': True,
 'team_a': 1,
 'team_a_score': 0,
 'team_h': 3,
 'team_h_score': 2,
 'stats': [{'identifier': 'goals_scored',
   'a': [],
   'h': [{'value': 1, 'element': 77}, {'value': 1, 'element': 81}]},
  {'identifier': 'assists', 'a': [], 'h': [{'value': 1, 'element': 91}]},
  {'identifier': 'own_goals', 'a': [], 'h': []},
  {'identifier': 'penalties_saved', 'a': [], 'h': []},
  {'identifier': 'penalties_missed', 'a': [], 'h': []},
  {'identifier': 'yellow_cards', 'a': [], 'h': []},
  {'identifier': 'red_cards', 'a': [], 'h': []},
  {'identifier': 'saves',
   'a': [{'value': 1, 'element': 1}],
   'h': [{'value': 4, 'element': 80}]},
  {'identifier': 'bonus',
   'a': [],
   'h': [{'value': 3, 'element': 81},
    {'value': 2, 'element': 91},
    {'value': 1, 'element': 80}]},
  {'identifier': 'bps',
  

In [8]:
fixtures[2]

{'code': 2210276,
 'event': 1,
 'finished': True,
 'finished_provisional': True,
 'id': 6,
 'kickoff_time': '2021-08-14T11:30:00Z',
 'minutes': 90,
 'provisional_start_time': False,
 'started': True,
 'team_a': 10,
 'team_a_score': 1,
 'team_h': 13,
 'team_h_score': 5,
 'stats': [{'identifier': 'goals_scored',
   'a': [{'value': 1, 'element': 185}],
   'h': [{'value': 3, 'element': 277},
    {'value': 1, 'element': 274},
    {'value': 1, 'element': 289}]},
  {'identifier': 'assists',
   'a': [{'value': 1, 'element': 188}],
   'h': [{'value': 4, 'element': 272}, {'value': 1, 'element': 284}]},
  {'identifier': 'own_goals', 'a': [], 'h': []},
  {'identifier': 'penalties_saved', 'a': [], 'h': []},
  {'identifier': 'penalties_missed', 'a': [], 'h': []},
  {'identifier': 'yellow_cards',
   'a': [{'value': 1, 'element': 184}, {'value': 1, 'element': 196}],
   'h': [{'value': 1, 'element': 275}]},
  {'identifier': 'red_cards', 'a': [], 'h': []},
  {'identifier': 'saves',
   'a': [{'value': 3,

In [ ]:
season_scores = pd.DataFrame()